In [5]:
import tkinter as tk
from tkinter import filedialog, Text, Button, Label, Scale, HORIZONTAL
from PIL import Image, ImageTk
import cv2
import os

is_paused = False
cap = None
emotion_at_time = []
current_emotion_label = None

import os
import sys
import time
import math
import glob
import copy
import warnings
import csv
import wave
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.fftpack import fft
from scipy.fftpack.realtransforms import dct
from scipy.signal import lfilter
from scipy.signal.windows import hamming
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.models as models

from transformers import T5EncoderModel, T5Config
from features import *
from helper import *

warnings.filterwarnings('ignore')


def f(tar):
    global emotion_at_time
    def get_audio(path_to_wav, filename):
        wav = wave.open(path_to_wav + filename, mode="r")
        (nchannels, sampwidth, framerate, nframes, comptype, compname) = wav.getparams()
        content = wav.readframes(nframes)
        samples = np.fromstring(content, dtype=np.int16)
        return (nchannels, sampwidth, framerate, nframes, comptype, compname), samples
    def get_transcriptions(path_to_transcriptions, filename):
        f = open(path_to_transcriptions + filename, 'r').read()
        f = np.array(f.split('\n'))
        transcription = {}
        for i in range(len(f) - 1):
            g = f[i]
            i1 = g.find(': ')
            i0 = g.find(' [')
            ind_id = g[:i0]
            ind_ts = g[i1+2:]
            transcription[ind_id] = ind_ts
        return transcription
    batch_size = 64
    nb_feat = 34
    nb_class = 4
    nb_epoch = 80

    optimizer = 'Adadelta'
    ts = []
    
    code_path = os.path.dirname(os.path.realpath(os.getcwd()))
    emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])
    data_path = "../../../data/sessions/"
    sessions = ['Session1','Session2','Session3','Session4','Session5']
    framerate = 16000

    def get_mocap_rot(path_to_mocap_rot, filename, start,end):
        f = open(path_to_mocap_rot + filename, 'r').read()
        f = np.array(f.split('\n'))
        mocap_rot = []
        mocap_rot_avg = []
        f = f[2:]
        counter = 0
        for data in f:
            counter+=1
            data2 = data.split(' ')
            if(len(data2)<2):
                continue
            if(float(data2[1])>start and float(data2[1])<end):
                mocap_rot_avg.append(np.array(data2[2:]).astype(float))

        mocap_rot_avg = np.array_split(np.array(mocap_rot_avg), 200)
        for spl in mocap_rot_avg:
            mocap_rot.append(np.mean(spl, axis=0))
        return np.array(mocap_rot)

    def get_mocap_hand(path_to_mocap_hand, filename, start,end):
        f = open(path_to_mocap_hand + filename, 'r').read()
        f = np.array(f.split('\n'))
        mocap_hand = []
        mocap_hand_avg = []
        f = f[2:]
        counter = 0
        for data in f:
            counter+=1
            data2 = data.split(' ')
            if(len(data2)<2):
                continue
            if(float(data2[1])>start and float(data2[1])<end):
                mocap_hand_avg.append(np.array(data2[2:]).astype(float))

        mocap_hand_avg = np.array_split(np.array(mocap_hand_avg), 200)
        for spl in mocap_hand_avg:
            mocap_hand.append(np.mean(spl, axis=0))
        return np.array(mocap_hand)

    def split_wav(wav, emotions):
        (nchannels, sampwidth, framerate, nframes, comptype, compname), samples = wav

        left = samples[0::nchannels]
        right = samples[1::nchannels]

        frames = []
        for ie, e in enumerate(emotions):
            start = e['start']
            end = e['end']

            e['right'] = right[int(start * framerate):int(end * framerate)]
            e['left'] = left[int(start * framerate):int(end * framerate)]

            frames.append({'left': e['left'], 'right': e['right']})
        return frames

    def get_emotions(path_to_emotions, filename):
        f = open(path_to_emotions + filename, 'r').read()
        f = np.array(f.split('\n'))
        idx = f == ''
        idx_n = np.arange(len(f))[idx]
        emotion = []
        for i in range(len(idx_n) - 2):
            g = f[idx_n[i]+1:idx_n[i+1]]
            head = g[0]
            i0 = head.find(' - ')
            start_time = float(head[head.find('[') + 1:head.find(' - ')])
            end_time = float(head[head.find(' - ') + 3:head.find(']')])
            actor_id = head[head.find(filename[:-4]) + len(filename[:-4]) + 1:
                            head.find(filename[:-4]) + len(filename[:-4]) + 5]
            emo = head[head.find('\t[') - 3:head.find('\t[')]
            vad = head[head.find('\t[') + 1:]

            v = float(vad[1:7])
            a = float(vad[9:15])
            d = float(vad[17:23])

            j = 1
            emos = []
            while g[j][0] == "C":
                head = g[j]
                start_idx = head.find("\t") + 1
                evoluator_emo = []
                idx = head.find(";", start_idx)
                while idx != -1:
                    evoluator_emo.append(head[start_idx:idx].strip().lower()[:3])
                    start_idx = idx + 1
                    idx = head.find(";", start_idx)
                emos.append(evoluator_emo)
                j += 1

            emotion.append({'start': start_time,
                            'end': end_time,
                            'id': filename[:-4] + '_' + actor_id,
                            'v': v,
                            'a': a,
                            'd': d,
                            'emotion': emo,
                            'emo_evo': emos})
        return emotion


    def get_mocap_head(path_to_mocap_head, filename, start,end):
        f = open(path_to_mocap_head + filename, 'r').read()
        f = np.array(f.split('\n'))
        mocap_head = []
        mocap_head_avg = []
        f = f[2:]
        counter = 0
        for data in f:
            counter+=1
            data2 = data.split(' ')
            if(len(data2)<2):
                continue
            if(float(data2[1])>start and float(data2[1])<end):
                mocap_head_avg.append(np.array(data2[2:]).astype(float))

        mocap_head_avg = np.array_split(np.array(mocap_head_avg), 200)
        for spl in mocap_head_avg:
            mocap_head.append(np.mean(spl, axis=0))
        return np.array(mocap_head)



    def get_field(data, key):
        return np.array([e[key] for e in data])

    def read_iemocap_mocap():
        data = []
        ids = {}
        for session in sessions:
            path_to_wav = data_path + session + '/dialog/wav/'
            path_to_emotions = data_path + session + '/dialog/EmoEvaluation/'
            path_to_transcriptions = data_path + session + '/dialog/transcriptions/'
            path_to_mocap_hand = data_path + session + '/dialog/MOCAP_hand/'
            path_to_mocap_rot = data_path + session + '/dialog/MOCAP_rotated/'
            path_to_mocap_head = data_path + session + '/dialog/MOCAP_head/'

            files2 = os.listdir(path_to_wav)

            files = []
            for f in files2:
                if f.endswith(".wav"):
                    if f[0] == '.':
                        files.append(f[2:-4])
                    else:
                        files.append(f[:-4])


            for f in files:
                if f == tar:
                    mocap_f = f
                    if (f== 'Ses05M_script01_1b'):
                        mocap_f = 'Ses05M_script01_1' 

                    wav = get_audio(path_to_wav, f + '.wav')
                    transcriptions = get_transcriptions(path_to_transcriptions, f + '.txt')
                    emotions = get_emotions(path_to_emotions, f + '.txt')

                    sample = split_wav(wav, emotions)

                    for ie, e in enumerate(emotions):
                        '''if 'F' in e['id']:
                            e['signal'] = sample[ie]['left']
                        else:
                            e['signal'] = sample[ie]['right']'''

                        e['signal'] = sample[ie]['left']
                        e.pop("left", None)
                        e.pop("right", None)
                        ts.append([e['start'], e['end']])
                        e['transcription'] = transcriptions[e['id']]
                        e['mocap_hand'] = get_mocap_hand(path_to_mocap_hand, mocap_f + '.txt', e['start'], e['end'])
                        e['mocap_rot'] = get_mocap_rot(path_to_mocap_rot, mocap_f + '.txt', e['start'], e['end'])
                        e['mocap_head'] = get_mocap_head(path_to_mocap_head, mocap_f + '.txt', e['start'], e['end'])
                        if e['emotion'] in emotions_used:
                            if e['id'] not in ids:
                                data.append(e)
                                ids[e['id']] = 1


        sort_key = get_field(data, "id")
        return np.array(data)[np.argsort(sort_key)]

    data = read_iemocap_mocap()

    import pickle
    with open( './result.pickle', 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)








    def pad_sequence_into_array(Xs, maxlen=None, truncating='post', padding='post', value=0.):

        Nsamples = len(Xs)
        if maxlen is None:
            lengths = [s.shape[0] for s in Xs]    # 'sequences' must be list, 's' must be numpy array, len(s) return the first dimension of s
            maxlen = np.max(lengths)

        Xout = np.ones(shape=[Nsamples, maxlen] + list(Xs[0].shape[1:]), dtype=Xs[0].dtype) * np.asarray(value, dtype=Xs[0].dtype)
        Mask = np.zeros(shape=[Nsamples, maxlen], dtype=Xout.dtype)
        for i in range(Nsamples):
            x = Xs[i]
            if truncating == 'pre':
                trunc = x[-maxlen:]
            elif truncating == 'post':
                trunc = x[:maxlen]
            else:
                raise ValueError("Truncating type '%s' not understood" % truncating)
            if padding == 'post':
                Xout[i, :len(trunc)] = trunc
                Mask[i, :len(trunc)] = 1
            elif padding == 'pre':
                Xout[i, -len(trunc):] = trunc
                Mask[i, -len(trunc):] = 1
            else:
                raise ValueError("Padding type '%s' not understood" % padding)
        return Xout, Mask


    def convert_gt_from_array_to_list(gt_batch, gt_batch_mask=None):

        B, L = gt_batch.shape
        gt_batch = gt_batch.astype('int')
        gts = []
        for i in range(B):
            if gt_batch_mask is None:
                l = L
            else:
                l = int(gt_batch_mask[i, :].sum())
            gts.append(gt_batch[i, :l].tolist())
        return gts










    eps = 0.00000001


    """ Time-domain audio features """


    def stZCR(frame):
        """Computes zero crossing rate of frame"""
        count = len(frame)
        countZ = numpy.sum(numpy.abs(numpy.diff(numpy.sign(frame)))) / 2
        return (numpy.float64(countZ) / numpy.float64(count-1.0))


    def stEnergy(frame):
        """Computes signal energy of frame"""
        return numpy.sum(frame ** 2) / numpy.float64(len(frame))


    def stEnergyEntropy(frame, numOfShortBlocks=10):
        """Computes entropy of energy"""
        Eol = numpy.sum(frame ** 2)    # total frame energy
        L = len(frame)
        subWinLength = int(numpy.floor(L / numOfShortBlocks))
        if L != subWinLength * numOfShortBlocks:
                frame = frame[0:subWinLength * numOfShortBlocks]
        # subWindows is of size [numOfShortBlocks x L]
        subWindows = frame.reshape(subWinLength, numOfShortBlocks, order='F').copy()

        # Compute normalized sub-frame energies:
        s = numpy.sum(subWindows ** 2, axis=0) / (Eol + eps)

        # Compute entropy of the normalized sub-frame energies:
        Entropy = -numpy.sum(s * numpy.log2(s + eps))
        return Entropy


    """ Frequency-domain audio features """


    def stSpectralCentroidAndSpread(X, fs):
        """Computes spectral centroid of frame (given abs(FFT))"""
        ind = (numpy.arange(1, len(X) + 1)) * (fs/(2.0 * len(X)))

        Xt = X.copy()
        Xt = Xt / Xt.max()
        NUM = numpy.sum(ind * Xt)
        DEN = numpy.sum(Xt) + eps

        # Centroid:
        C = (NUM / DEN)

        # Spread:
        S = numpy.sqrt(numpy.sum(((ind - C) ** 2) * Xt) / DEN)

        # Normalize:
        C = C / (fs / 2.0)
        S = S / (fs / 2.0)

        return (C, S)


    def stSpectralEntropy(X, numOfShortBlocks=10):
        """Computes the spectral entropy"""
        L = len(X)                         # number of frame samples
        Eol = numpy.sum(X ** 2)            # total spectral energy

        subWinLength = int(numpy.floor(L / numOfShortBlocks))   # length of sub-frame
        if L != subWinLength * numOfShortBlocks:
            X = X[0:subWinLength * numOfShortBlocks]

        subWindows = X.reshape(subWinLength, numOfShortBlocks, order='F').copy()  # define sub-frames (using matrix reshape)
        s = numpy.sum(subWindows ** 2, axis=0) / (Eol + eps)                      # compute spectral sub-energies
        En = -numpy.sum(s*numpy.log2(s + eps))                                    # compute spectral entropy

        return En


    def stSpectralFlux(X, Xprev):
        """
        Computes the spectral flux feature of the current frame
        ARGUMENTS:
            X:        the abs(fft) of the current frame
            Xpre:        the abs(fft) of the previous frame
        """
        # compute the spectral flux as the sum of square distances:
        sumX = numpy.sum(X + eps)
        sumPrevX = numpy.sum(Xprev + eps)
        F = numpy.sum((X / sumX - Xprev/sumPrevX) ** 2)

        return F


    def stSpectralRollOff(X, c, fs):
        """Computes spectral roll-off"""
        totalEnergy = numpy.sum(X ** 2)
        fftLength = len(X)
        Thres = c*totalEnergy
        # Ffind the spectral rolloff as the frequency position where the respective spectral energy is equal to c*totalEnergy
        CumSum = numpy.cumsum(X ** 2) + eps
        [a, ] = numpy.nonzero(CumSum > Thres)
        if len(a) > 0:
            mC = numpy.float64(a[0]) / (float(fftLength))
        else:
            mC = 0.0
        return (mC)


    def stHarmonic(frame, fs):
        """
        Computes harmonic ratio and pitch
        """
        M = numpy.round(0.016 * fs) - 1
        R = numpy.correlate(frame, frame, mode='full')

        g = R[len(frame)-1]
        R = R[len(frame):-1]

        # estimate m0 (as the first zero crossing of R)
        [a, ] = numpy.nonzero(numpy.diff(numpy.sign(R)))

        if len(a) == 0:
            m0 = len(R)-1
        else:
            m0 = a[0]
        if M > len(R):
            M = len(R) - 1

        Gamma = numpy.zeros((M), dtype=numpy.float64)
        CSum = numpy.cumsum(frame ** 2)
        Gamma[m0:M] = R[m0:M] / (numpy.sqrt((g * CSum[M:m0:-1])) + eps)

        ZCR = stZCR(Gamma)

        if ZCR > 0.15:
            HR = 0.0
            f0 = 0.0
        else:
            if len(Gamma) == 0:
                HR = 1.0
                blag = 0.0
                Gamma = numpy.zeros((M), dtype=numpy.float64)
            else:
                HR = numpy.max(Gamma)
                blag = numpy.argmax(Gamma)

            # Get fundamental frequency:
            f0 = fs / (blag + eps)
            if f0 > 5000:
                f0 = 0.0
            if HR < 0.1:
                f0 = 0.0

        return (HR, f0)


    def mfccInitFilterBanks(fs, nfft):
        """
        Computes the triangular filterbank for MFCC computation (used in the stFeatureExtraction function before the stMFCC function call)
        This function is taken from the scikits.talkbox library (MIT Licence):
        https://pypi.python.org/pypi/scikits.talkbox
        """

        # filter bank params:
        lowfreq = 133.33
        linsc = 200/3.
        logsc = 1.0711703
        numLinFiltTotal = 13
        numLogFilt = 27

        if fs < 8000:
            nlogfil = 5

        # Total number of filters
        nFiltTotal = numLinFiltTotal + numLogFilt
        #print (str(nFiltTotal))
        #print (str(nfft))

        # Compute frequency points of the triangle:
        freqs = numpy.zeros(nFiltTotal+2)
        freqs[:numLinFiltTotal] = lowfreq + numpy.arange(numLinFiltTotal) * linsc
        freqs[numLinFiltTotal:] = freqs[numLinFiltTotal-1] * logsc ** numpy.arange(1, numLogFilt + 3)
        heights = 2./(freqs[2:] - freqs[0:-2])

        # Compute filterbank coeff (in fft domain, in bins)


        fbank = numpy.zeros((int(nFiltTotal), int(nfft)))
        nfreqs = numpy.arange(nfft) / (1. * nfft) * fs

        for i in range(nFiltTotal):
            lowTrFreq = freqs[i]
            cenTrFreq = freqs[i+1]
            highTrFreq = freqs[i+2]

            lid = numpy.arange(numpy.floor(lowTrFreq * nfft / fs) + 1, numpy.floor(cenTrFreq * nfft / fs) + 1, dtype=int)
            lslope = heights[i] / (cenTrFreq - lowTrFreq)
            rid = numpy.arange(numpy.floor(cenTrFreq * nfft / fs) + 1, numpy.floor(highTrFreq * nfft / fs) + 1, dtype=int)
            rslope = heights[i] / (highTrFreq - cenTrFreq)
            fbank[i][lid] = lslope * (nfreqs[lid] - lowTrFreq)
            fbank[i][rid] = rslope * (highTrFreq - nfreqs[rid])

        return fbank, freqs


    def stMFCC(X, fbank, nceps):
        """
        Computes the MFCCs of a frame, given the fft mag
        ARGUMENTS:
            X:        fft magnitude abs(FFT)
            fbank:    filter bank (see mfccInitFilterBanks)
        RETURN
            ceps:     MFCCs (13 element vector)
        Note:    MFCC calculation is, in general, taken from the scikits.talkbox library (MIT Licence),
        #    with a small number of modifications to make it more compact and suitable for the pyAudioAnalysis Lib
        """

        mspec = numpy.log10(numpy.dot(X, fbank.T)+eps)
        ceps = dct(mspec, type=2, norm='ortho', axis=-1)[:nceps]
        return ceps


    def stChromaFeaturesInit(nfft, fs):
        """
        This function initializes the chroma matrices used in the calculation of the chroma features
        """
        freqs = numpy.array([((f + 1) * fs) / (2 * int(nfft)) for f in range(int(nfft))])
        Cp = 27.50

        nChroma = numpy.round(12.0 * numpy.log2(freqs / Cp)).astype(int)

        nFreqsPerChroma = numpy.zeros((nChroma.shape[0], ))

        uChroma = numpy.unique(nChroma)
        for u in uChroma:
            idx = numpy.nonzero(nChroma == u)
            nFreqsPerChroma[idx] = idx[0].shape
        return nChroma, nFreqsPerChroma


    def stChromaFeatures(X, fs, nChroma, nFreqsPerChroma):
        #TODO: 1 complexity
        #TODO: 2 bug with large windows

        chromaNames = ['A', 'A#', 'B', 'C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#']
        spec = X**2
        C = numpy.zeros((nChroma.shape[0],))
        C[nChroma] = spec
        C /= nFreqsPerChroma[nChroma]
        finalC = numpy.zeros((12, 1))
        newD = int(numpy.ceil(C.shape[0] // 12.0) * 12)
        C2 = numpy.zeros((newD, ))
        #C2[0:C.shape[0]] = C
        C2 = C2.reshape(int(numpy.ceil(C2.shape[0]//12)), 12)
        #for i in range(12):
        #    finalC[i] = numpy.sum(C[i:C.shape[0]:12])
        finalC = numpy.matrix(numpy.sum(C2, axis=0)).T
        finalC /= spec.sum()

    #    ax = plt.gca()
    #    plt.hold(False)
    #    plt.plot(finalC)
    #    ax.set_xticks(range(len(chromaNames)))
    #    ax.set_xticklabels(chromaNames)
    #    xaxis = numpy.arange(0, 0.02, 0.01);
    #    ax.set_yticks(range(len(xaxis)))
    #    ax.set_yticklabels(xaxis)
    #    plt.show(block=False)
    #    plt.draw()

        return chromaNames, finalC


    def stChromagram(signal, Fs, Win, Step, PLOT=False):
        """
        Short-term FFT mag for spectogram estimation:
        Returns:
            a numpy array (nFFT x numOfShortTermWindows)
        ARGUMENTS:
            signal:      the input signal samples
            Fs:          the sampling freq (in Hz)
            Win:         the short-term window size (in samples)
            Step:        the short-term window step (in samples)
            PLOT:        flag, 1 if results are to be ploted
        RETURNS:
        """
        Win = int(Win)
        Step = int(Step)
        signal = numpy.double(signal)
        signal = signal / (2.0 ** 15)
        DC = signal.mean()
        MAX = (numpy.abs(signal)).max()
        signal = (signal - DC) / (MAX - DC)

        N = len(signal)        # total number of signals
        curPos = 0
        countFrames = 0
        nfft = int(Win / 2)
        nChroma, nFreqsPerChroma = stChromaFeaturesInit(nfft, Fs)
        chromaGram = numpy.array([], dtype=numpy.float64)

        while (curPos + Win - 1 < N):
            countFrames += 1
            x = signal[curPos:curPos + Win]
            curPos = curPos + Step
            X = abs(fft(x))
            X = X[0:nfft]
            X = X / len(X)
            chromaNames, C = stChromaFeatures(X, Fs, nChroma, nFreqsPerChroma)
            C = C[:, 0]
            if countFrames == 1:
                chromaGram = C.T
            else:
                chromaGram = numpy.vstack((chromaGram, C.T))
        FreqAxis = chromaNames
        TimeAxis = [(t * Step) / Fs for t in range(chromaGram.shape[0])]

        if (PLOT):
            fig, ax = plt.subplots()
            chromaGramToPlot = chromaGram.transpose()[::-1, :]
            Ratio = chromaGramToPlot.shape[1] / (3*chromaGramToPlot.shape[0])
            chromaGramToPlot = numpy.repeat(chromaGramToPlot, Ratio, axis=0)
            imgplot = plt.imshow(chromaGramToPlot)
            Fstep = int(nfft / 5.0)
    #        FreqTicks = range(0, int(nfft) + Fstep, Fstep)
    #        FreqTicksLabels = [str(Fs/2-int((f*Fs) / (2*nfft))) for f in FreqTicks]
            ax.set_yticks(range(Ratio / 2, len(FreqAxis) * Ratio, Ratio))
            ax.set_yticklabels(FreqAxis[::-1])
            TStep = countFrames / 3
            TimeTicks = range(0, countFrames, TStep)
            TimeTicksLabels = ['%.2f' % (float(t * Step) / Fs) for t in TimeTicks]
            ax.set_xticks(TimeTicks)
            ax.set_xticklabels(TimeTicksLabels)
            ax.set_xlabel('time (secs)')
            imgplot.set_cmap('jet')
            plt.colorbar()
            plt.show()

        return (chromaGram, TimeAxis, FreqAxis)


    def phormants(x, Fs):
        N = len(x)
        w = numpy.hamming(N)

        # Apply window and high pass filter.
        x1 = x * w   
        x1 = lfilter([1], [1., 0.63], x1)

        # Get LPC.    
        ncoeff = 2 + Fs / 1000
        A, e, k = lpc(x1, ncoeff)    
        #A, e, k = lpc(x1, 8)

        # Get roots.
        rts = numpy.roots(A)
        rts = [r for r in rts if numpy.imag(r) >= 0]

        # Get angles.
        angz = numpy.arctan2(numpy.imag(rts), numpy.real(rts))

        # Get frequencies.    
        frqs = sorted(angz * (Fs / (2 * math.pi)))

        return frqs
    def beatExtraction(stFeatures, winSize, PLOT=False):
        """
        This function extracts an estimate of the beat rate for a musical signal.
        ARGUMENTS:
         - stFeatures:     a numpy array (numOfFeatures x numOfShortTermWindows)
         - winSize:        window size in seconds
        RETURNS:
         - BPM:            estimates of beats per minute
         - Ratio:          a confidence measure
        """

        # Features that are related to the beat tracking task:
        toWatch = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

        maxBeatTime = int(round(2.0 / winSize))
        HistAll = numpy.zeros((maxBeatTime,))
        for ii, i in enumerate(toWatch):                                        # for each feature
            DifThres = 2.0 * (numpy.abs(stFeatures[i, 0:-1] - stFeatures[i, 1::])).mean()    # dif threshold (3 x Mean of Difs)
            [pos1, _] = utilities.peakdet(stFeatures[i, :], DifThres)           # detect local maxima
            posDifs = []                                                        # compute histograms of local maxima changes
            for j in range(len(pos1)-1):
                posDifs.append(pos1[j+1]-pos1[j])
            [HistTimes, HistEdges] = numpy.histogram(posDifs, numpy.arange(0.5, maxBeatTime + 1.5))
            HistCenters = (HistEdges[0:-1] + HistEdges[1::]) / 2.0
            HistTimes = HistTimes.astype(float) / stFeatures.shape[1]
            HistAll += HistTimes
            if PLOT:
                plt.subplot(9, 2, ii + 1)
                plt.plot(stFeatures[i, :], 'k')
                for k in pos1:
                    plt.plot(k, stFeatures[i, k], 'k*')
                f1 = plt.gca()
                f1.axes.get_xaxis().set_ticks([])
                f1.axes.get_yaxis().set_ticks([])

        if PLOT:
            plt.show(block=False)
            plt.figure()

        # Get beat as the argmax of the agregated histogram:
        I = numpy.argmax(HistAll)
        BPMs = 60 / (HistCenters * winSize)
        BPM = BPMs[I]
        # ... and the beat ratio:
        Ratio = HistAll[I] / HistAll.sum()

        if PLOT:
            # filter out >500 beats from plotting:
            HistAll = HistAll[BPMs < 500]
            BPMs = BPMs[BPMs < 500]

            plt.plot(BPMs, HistAll, 'k')
            plt.xlabel('Beats per minute')
            plt.ylabel('Freq Count')
            plt.show(block=True)

        return BPM, Ratio


    def stSpectogram(signal, Fs, Win, Step, PLOT=False):
        """
        Short-term FFT mag for spectogram estimation:
        Returns:
            a numpy array (nFFT x numOfShortTermWindows)
        ARGUMENTS:
            signal:      the input signal samples
            Fs:          the sampling freq (in Hz)
            Win:         the short-term window size (in samples)
            Step:        the short-term window step (in samples)
            PLOT:        flag, 1 if results are to be ploted
        RETURNS:
        """
        Win = int(Win)
        Step = int(Step)
        signal = numpy.double(signal)
        signal = signal / (2.0 ** 15)
        DC = signal.mean()
        MAX = (numpy.abs(signal)).max()
        signal = (signal - DC) / (MAX - DC)

        N = len(signal)        # total number of signals
        curPos = 0
        countFrames = 0
        nfft = int(Win / 2)
        specgram = numpy.array([], dtype=numpy.float64)

        while (curPos + Win - 1 < N):
            countFrames += 1
            x = signal[curPos:curPos+Win]
            curPos = curPos + Step
            X = abs(fft(x))
            X = X[0:nfft]
            X = X / len(X)

            if countFrames == 1:
                specgram = X ** 2
            else:
                specgram = numpy.vstack((specgram, X))

        FreqAxis = [((f + 1) * Fs) / (2 * nfft) for f in range(specgram.shape[1])]
        TimeAxis = [(t * Step) / Fs for t in range(specgram.shape[0])]

        if (PLOT):
            fig, ax = plt.subplots()
            imgplot = plt.imshow(specgram.transpose()[::-1, :])
            Fstep = int(nfft / 5.0)
            FreqTicks = range(0, int(nfft) + Fstep, Fstep)
            FreqTicksLabels = [str(Fs / 2 - int((f * Fs) / (2 * nfft))) for f in FreqTicks]
            ax.set_yticks(FreqTicks)
            ax.set_yticklabels(FreqTicksLabels)
            TStep = countFrames/3
            TimeTicks = range(0, countFrames, TStep)
            TimeTicksLabels = ['%.2f' % (float(t * Step) / Fs) for t in TimeTicks]
            ax.set_xticks(TimeTicks)
            ax.set_xticklabels(TimeTicksLabels)
            ax.set_xlabel('time (secs)')
            ax.set_ylabel('freq (Hz)')
            imgplot.set_cmap('jet')
            plt.colorbar()
            plt.show()

        return (specgram, TimeAxis, FreqAxis)


    """ Windowing and feature extraction """


    def stFeatureExtraction(signal, Fs, Win, Step):
        """
        This function implements the shor-term windowing process. For each short-term window a set of features is extracted.
        This results to a sequence of feature vectors, stored in a numpy matrix.
        ARGUMENTS
            signal:       the input signal samples
            Fs:           the sampling freq (in Hz)
            Win:          the short-term window size (in samples)
            Step:         the short-term window step (in samples)
        RETURNS
            stFeatures:   a numpy array (numOfFeatures x numOfShortTermWindows)
        """

        Win = int(Win)
        Step = int(Step)

        # Signal normalization
        signal = numpy.double(signal)

        signal = signal / (2.0 ** 15)
        DC = signal.mean()
        MAX = (numpy.abs(signal)).max()
        signal = (signal - DC) / MAX

        N = len(signal)                                # total number of samples
        curPos = 0
        countFrames = 0
        nFFT = Win / 2

        [fbank, freqs] = mfccInitFilterBanks(Fs, nFFT)                # compute the triangular filter banks used in the mfcc calculation
        nChroma, nFreqsPerChroma = stChromaFeaturesInit(nFFT, Fs)


        numOfPitches = 5
        numOfPeaks = 10
        numOfTimeSpectralFeatures = 8
        numOfHarmonicFeatures = 0
        nceps = 13
        numOfChromaFeatures = 13

        totalNumOfFeatures = numOfTimeSpectralFeatures + nceps + numOfHarmonicFeatures + numOfChromaFeatures
    #    totalNumOfFeatures = numOfTimeSpectralFeatures + nceps + numOfHarmonicFeatures
        stFeatures = numpy.array([], dtype=numpy.float64)

        while (curPos + Win - 1 < N):                        # for each short-term window until the end of signal
            countFrames += 1
            x = signal[curPos:curPos+Win]                    # get current window
            curPos = curPos + Step                           # update window position
            X = abs(fft(x))                                  # get fft magnitude
            X = X[0:int(nFFT)]                                    # normalize fft
            X = X / len(X)
            if countFrames == 1:
                Xprev = X.copy()                             # keep previous fft mag (used in spectral flux)
            curFV = numpy.zeros((totalNumOfFeatures, 1))
            curFV[0] = stZCR(x)                              # zero crossing rate
            curFV[1] = stEnergy(x)                           # short-term energy
            curFV[2] = stEnergyEntropy(x)                    # short-term entropy of energy
            [curFV[3], curFV[4]] = stSpectralCentroidAndSpread(X, Fs)    # spectral centroid and spread
            curFV[5] = stSpectralEntropy(X)                  # spectral entropy
            curFV[6] = stSpectralFlux(X, Xprev)              # spectral flux
            curFV[7] = stSpectralRollOff(X, 0.90, Fs)        # spectral rolloff
            curFV[numOfTimeSpectralFeatures:numOfTimeSpectralFeatures+nceps, 0] = stMFCC(X, fbank, nceps).copy()    # MFCCs

            chromaNames, chromaF = stChromaFeatures(X, Fs, nChroma, nFreqsPerChroma)
            curFV[numOfTimeSpectralFeatures + nceps: numOfTimeSpectralFeatures + nceps + numOfChromaFeatures - 1] = chromaF
            numOfCFFeatures = numOfTimeSpectralFeatures + nceps + numOfChromaFeatures

            curFV[numOfCFFeatures-1] = chromaF.std()

            if countFrames == 1:
                stFeatures = curFV                                        # initialize feature matrix (if first frame)
            else:
                stFeatures = numpy.concatenate((stFeatures, curFV), 1)    # update feature matrix
            Xprev = X.copy()

        return numpy.array(stFeatures)


    def mtFeatureExtraction(signal, Fs, mtWin, mtStep, stWin, stStep):
        """
        Mid-term feature extraction
        """

        mtWinRatio = int(round(mtWin / stStep))
        mtStepRatio = int(round(mtStep / stStep))

        mtFeatures = []

        stFeatures = stFeatureExtraction(signal, Fs, stWin, stStep)
        numOfFeatures = len(stFeatures)
        numOfStatistics = 2

        mtFeatures = []
        #for i in range(numOfStatistics * numOfFeatures + 1):
        for i in range(numOfStatistics * numOfFeatures):
            mtFeatures.append([])

        for i in range(numOfFeatures):        # for each of the short-term features:
            curPos = 0
            N = len(stFeatures[i])
            while (curPos < N):
                N1 = curPos
                N2 = curPos + mtWinRatio
                if N2 > N:
                    N2 = N
                curStFeatures = stFeatures[i][N1:N2]

                mtFeatures[i].append(numpy.mean(curStFeatures))
                mtFeatures[i+numOfFeatures].append(numpy.std(curStFeatures))
                #mtFeatures[i+2*numOfFeatures].append(numpy.std(curStFeatures) / (numpy.mean(curStFeatures)+0.00000010))
                curPos += mtStepRatio

        return numpy.array(mtFeatures), stFeatures


    # TODO
    def stFeatureSpeed(signal, Fs, Win, Step):

        signal = numpy.double(signal)
        signal = signal / (2.0 ** 15)
        DC = signal.mean()
        MAX = (numpy.abs(signal)).max()
        signal = (signal - DC) / MAX
        # print (numpy.abs(signal)).max()

        N = len(signal)        # total number of signals
        curPos = 0
        countFrames = 0

        lowfreq = 133.33
        linsc = 200/3.
        logsc = 1.0711703
        nlinfil = 13
        nlogfil = 27
        nceps = 13
        nfil = nlinfil + nlogfil
        nfft = Win / 2
        if Fs < 8000:
            nlogfil = 5
            nfil = nlinfil + nlogfil
            nfft = Win / 2

        # compute filter banks for mfcc:
        [fbank, freqs] = mfccInitFilterBanks(Fs, nfft, lowfreq, linsc, logsc, nlinfil, nlogfil)

        numOfTimeSpectralFeatures = 8
        numOfHarmonicFeatures = 1
        totalNumOfFeatures = numOfTimeSpectralFeatures + nceps + numOfHarmonicFeatures
        #stFeatures = numpy.array([], dtype=numpy.float64)
        stFeatures = []

        while (curPos + Win - 1 < N):
            countFrames += 1
            x = signal[curPos:curPos + Win]
            curPos = curPos + Step
            X = abs(fft(x))
            X = X[0:nfft]
            X = X / len(X)
            Ex = 0.0
            El = 0.0
            X[0:4] = 0
    #        M = numpy.round(0.016 * fs) - 1
    #        R = numpy.correlate(frame, frame, mode='full')
            stFeatures.append(stHarmonic(x, Fs))
    #        for i in range(len(X)):
                #if (i < (len(X) / 8)) and (i > (len(X)/40)):
                #    Ex += X[i]*X[i]
                #El += X[i]*X[i]
    #        stFeatures.append(Ex / El)
    #        stFeatures.append(numpy.argmax(X))
    #        if curFV[numOfTimeSpectralFeatures+nceps+1]>0:
    #            print curFV[numOfTimeSpectralFeatures+nceps], curFV[numOfTimeSpectralFeatures+nceps+1]
        return numpy.array(stFeatures)


    """ Feature Extraction Wrappers
     - The first two feature extraction wrappers are used to extract long-term averaged
       audio features for a list of WAV files stored in a given category.
       It is important to note that, one single feature is extracted per WAV file (not the whole sequence of feature vectors)
     """


    def dirWavFeatureExtraction(dirName, mtWin, mtStep, stWin, stStep, computeBEAT=False):
        """
        This function extracts the mid-term features of the WAVE files of a particular folder.
        The resulting feature vector is extracted by long-term averaging the mid-term features.
        Therefore ONE FEATURE VECTOR is extracted for each WAV file.
        ARGUMENTS:
            - dirName:        the path of the WAVE directory
            - mtWin, mtStep:    mid-term window and step (in seconds)
            - stWin, stStep:    short-term window and step (in seconds)
        """

        allMtFeatures = numpy.array([])
        processingTimes = []

        types = ('*.wav', '*.aif',  '*.aiff')
        wavFilesList = []
        for files in types:
            wavFilesList.extend(glob.glob(os.path.join(dirName, files)))

        wavFilesList = sorted(wavFilesList)

        for wavFile in wavFilesList:
            [Fs, x] = audioBasicIO.readAudioFile(wavFile)            # read file
            t1 = time.clock()
            x = audioBasicIO.stereo2mono(x)                          # convert stereo to mono
            if computeBEAT:                                          # mid-term feature extraction for current file
                [MidTermFeatures, stFeatures] = mtFeatureExtraction(x, Fs, round(mtWin * Fs), round(mtStep * Fs), round(Fs * stWin), round(Fs * stStep))
                [beat, beatConf] = beatExtraction(stFeatures, stStep)
            else:
                [MidTermFeatures, _] = mtFeatureExtraction(x, Fs, round(mtWin * Fs), round(mtStep * Fs), round(Fs * stWin), round(Fs * stStep))

            MidTermFeatures = numpy.transpose(MidTermFeatures)
            MidTermFeatures = MidTermFeatures.mean(axis=0)         # long term averaging of mid-term statistics
            if computeBEAT:
                MidTermFeatures = numpy.append(MidTermFeatures, beat)
                MidTermFeatures = numpy.append(MidTermFeatures, beatConf)
            if len(allMtFeatures) == 0:                              # append feature vector
                allMtFeatures = MidTermFeatures
            else:
                allMtFeatures = numpy.vstack((allMtFeatures, MidTermFeatures))
            t2 = time.clock()
            duration = float(len(x)) / Fs
            processingTimes.append((t2 - t1) / duration)
        if len(processingTimes) > 0:
            print("Feature extraction complexity ratio: {0:.1f} x realtime".format((1.0 / numpy.mean(numpy.array(processingTimes)))))
        return (allMtFeatures, wavFilesList)


    def dirsWavFeatureExtraction(dirNames, mtWin, mtStep, stWin, stStep, computeBEAT=False):
        '''
        Same as dirWavFeatureExtraction, but instead of a single dir it takes a list of paths as input and returns a list of feature matrices.
        EXAMPLE:
        [features, classNames] =
               a.dirsWavFeatureExtraction(['audioData/classSegmentsRec/noise','audioData/classSegmentsRec/speech',
                                           'audioData/classSegmentsRec/brush-teeth','audioData/classSegmentsRec/shower'], 1, 1, 0.02, 0.02);
        It can be used during the training process of a classification model ,
        in order to get feature matrices from various audio classes (each stored in a seperate path)
        '''

        # feature extraction for each class:
        features = []
        classNames = []
        fileNames = []
        for i, d in enumerate(dirNames):
            [f, fn] = dirWavFeatureExtraction(d, mtWin, mtStep, stWin, stStep, computeBEAT=computeBEAT)
            if f.shape[0] > 0:       # if at least one audio file has been found in the provided folder:
                features.append(f)
                fileNames.append(fn)
                if d[-1] == "/":
                    classNames.append(d.split(os.sep)[-2])
                else:
                    classNames.append(d.split(os.sep)[-1])
        return features, classNames, fileNames


    def dirWavFeatureExtractionNoAveraging(dirName, mtWin, mtStep, stWin, stStep):
        """
        This function extracts the mid-term features of the WAVE files of a particular folder without averaging each file.
        ARGUMENTS:
            - dirName:          the path of the WAVE directory
            - mtWin, mtStep:    mid-term window and step (in seconds)
            - stWin, stStep:    short-term window and step (in seconds)
        RETURNS:
            - X:                A feature matrix
            - Y:                A matrix of file labels
            - filenames:
        """

        allMtFeatures = numpy.array([])
        signalIndices = numpy.array([])
        processingTimes = []

        types = ('*.wav', '*.aif',  '*.aiff')
        wavFilesList = []
        for files in types:
            wavFilesList.extend(glob.glob(os.path.join(dirName, files)))

        wavFilesList = sorted(wavFilesList)

        for i, wavFile in enumerate(wavFilesList):
            [Fs, x] = audioBasicIO.readAudioFile(wavFile)            # read file
            x = audioBasicIO.stereo2mono(x)                          # convert stereo to mono
            [MidTermFeatures, _] = mtFeatureExtraction(x, Fs, round(mtWin * Fs), round(mtStep * Fs), round(Fs * stWin), round(Fs * stStep))  # mid-term feature

            MidTermFeatures = numpy.transpose(MidTermFeatures)
    #        MidTermFeatures = MidTermFeatures.mean(axis=0)        # long term averaging of mid-term statistics
            if len(allMtFeatures) == 0:                # append feature vector
                allMtFeatures = MidTermFeatures
                signalIndices = numpy.zeros((MidTermFeatures.shape[0], ))
            else:
                allMtFeatures = numpy.vstack((allMtFeatures, MidTermFeatures))
                signalIndices = numpy.append(signalIndices, i * numpy.ones((MidTermFeatures.shape[0], )))

        return (allMtFeatures, signalIndices, wavFilesList)



    def mtFeatureExtractionToFile(fileName, midTermSize, midTermStep, shortTermSize, shortTermStep, outPutFile,
                                  storeStFeatures=False, storeToCSV=False, PLOT=False):
        """
        This function is used as a wrapper to:
        a) read the content of a WAV file
        b) perform mid-term feature extraction on that signal
        c) write the mid-term feature sequences to a numpy file
        """
        [Fs, x] = audioBasicIO.readAudioFile(fileName)            # read the wav file
        x = audioBasicIO.stereo2mono(x)                           # convert to MONO if required
        if storeStFeatures:
            [mtF, stF] = mtFeatureExtraction(x, Fs, round(Fs * midTermSize), round(Fs * midTermStep), round(Fs * shortTermSize), round(Fs * shortTermStep))
        else:
            [mtF, _] = mtFeatureExtraction(x, Fs, round(Fs*midTermSize), round(Fs * midTermStep), round(Fs * shortTermSize), round(Fs * shortTermStep))

        numpy.save(outPutFile, mtF)                              # save mt features to numpy file
        if PLOT:
            print("Mid-term numpy file: " + outPutFile + ".npy saved")
        if storeToCSV:
            numpy.savetxt(outPutFile+".csv", mtF.T, delimiter=",")
            if PLOT:
                print("Mid-term CSV file: " + outPutFile + ".csv saved")

        if storeStFeatures:
            numpy.save(outPutFile+"_st", stF)                    # save st features to numpy file
            if PLOT:
                print("Short-term numpy file: " + outPutFile + "_st.npy saved")
            if storeToCSV:
                numpy.savetxt(outPutFile+"_st.csv", stF.T, delimiter=",")    # store st features to CSV file
                if PLOT:
                    print("Short-term CSV file: " + outPutFile + "_st.csv saved")


    def mtFeatureExtractionToFileDir(dirName, midTermSize, midTermStep, shortTermSize, shortTermStep, storeStFeatures=False, storeToCSV=False, PLOT=False):
        types = (dirName + os.sep + '*.wav', )
        filesToProcess = []
        for files in types:
            filesToProcess.extend(glob.glob(files))
        for f in filesToProcess:
            outPath = f
            mtFeatureExtractionToFile(f, midTermSize, midTermStep, shortTermSize, shortTermStep, outPath, storeStFeatures, storeToCSV, PLOT)


    # #### 

    # In[3]:


    code_path = os.path.dirname(os.path.realpath(os.getcwd()))
    emotions_used = np.array(['ang', 'exc', 'neu', 'sad'])

    framerate = 16000


    # In[5]:


    import pickle
    with open('./result.pickle', 'rb') as handle:
        data2 = pickle.load(handle)


    # In[6]:


    def calculate_features(frames, freq, options):
        window_sec = 0.2
        window_n = int(freq * window_sec)

        st_f = stFeatureExtraction(frames, freq, window_n, window_n / 2)

        if st_f.shape[1] > 2:
            i0 = 1
            i1 = st_f.shape[1] - 1
            if i1 - i0 < 1:
                i1 = i0 + 1

            deriv_st_f = np.zeros((st_f.shape[0], i1 - i0), dtype=float)
            for i in range(i0, i1):
                i_left = i - 1
                i_right = i + 1
                deriv_st_f[:st_f.shape[0], i - i0] = st_f[:, i]
            return deriv_st_f
        elif st_f.shape[1] == 2:
            deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
            deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
            return deriv_st_f
        else:
            deriv_st_f = np.zeros((st_f.shape[0], 1), dtype=float)
            deriv_st_f[:st_f.shape[0], 0] = st_f[:, 0]
            return deriv_st_f



    x_train_speech = []

    counter = 0
    for ses_mod in data2:
        x_head = ses_mod['signal']
        st_features = calculate_features(x_head, framerate, None)
        st_features, _ = pad_sequence_into_array(st_features, maxlen=100)
        x_train_speech.append( st_features.T )
        counter+=1


    x_train_speech = np.array(x_train_speech)


    x_train_mocap = []
    counter = 0
    for ses_mod in data2:
        x_head = ses_mod['mocap_head']
        if(x_head.shape != (200,18)):
            x_head = np.zeros((200,18))   
        x_head[np.isnan(x_head)]=0
        x_hand = ses_mod['mocap_hand']
        if(x_hand.shape != (200,6)):
            x_hand = np.zeros((200,6))   
        x_hand[np.isnan(x_hand)]=0
        x_rot = ses_mod['mocap_rot']
        if(x_rot.shape != (200,165)):
            x_rot = np.zeros((200,165))   
        x_rot[np.isnan(x_rot)]=0
        x_mocap = np.concatenate((x_head, x_hand), axis=1)
        x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
        x_train_mocap.append(x_mocap)

    x_train_mocap = np.array(x_train_mocap)
    x_train_mocap = x_train_mocap.reshape(-1,200,189,1)





    Y=[]
    for ses_mod in data2:
        Y.append(ses_mod['emotion'])
    Y = label_binarize(Y,classes=emotions_used)



    class_names = ['ang', 'exc', 'neu', 'sad']
    n_classes = len(class_names)

    class CrossAttentionBlock(nn.Module):
        def __init__(self, embed_dim, num_heads):
            super(CrossAttentionBlock, self).__init__()
            self.cross_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
            self.norm = nn.LayerNorm(embed_dim)
            self.dropout = nn.Dropout(0.1)

        def forward(self, query, key_value):
            attn_output, _ = self.cross_attn(query, key_value, key_value)
            out = self.norm(query + self.dropout(attn_output))
            return out

    class CombinedModel(nn.Module):
        def __init__(self):
            super(CombinedModel, self).__init__()
            self.speech_model = SpeechModel()
            self.mocap_model = MoCapModel()
            self.cross_attn_s2m = CrossAttentionBlock(embed_dim=256, num_heads=4)
            self.cross_attn_m2s = CrossAttentionBlock(embed_dim=256, num_heads=4)
            self.fc1 = nn.Linear(512, 256)
            encoder_layer = nn.TransformerEncoderLayer(d_model=256, nhead=4, batch_first=True)
            self.fusion_transformer = nn.TransformerEncoder(encoder_layer, num_layers=1)
            self.fc2 = nn.Linear(256, 4)

        def forward(self, x_speech, x_mocap):
            out_speech = self.speech_model(x_speech)
            out_mocap = self.mocap_model(x_mocap)
            out_speech = out_speech.unsqueeze(1)
            out_mocap = out_mocap.unsqueeze(1)
            speech_attended = self.cross_attn_s2m(out_speech, out_mocap)
            mocap_attended = self.cross_attn_m2s(out_mocap, out_speech)
            combined = torch.cat((speech_attended, mocap_attended), dim=-1).squeeze(1)
            x = F.relu(self.fc1(combined)).unsqueeze(1)
            x = self.fusion_transformer(x).squeeze(1)
            x = self.fc2(x)
            return x

    class SpeechModel(nn.Module):
        def __init__(self):
            super(SpeechModel, self).__init__()
            config = T5Config(
                d_model=128,
                d_kv=32,
                d_ff=256,
                num_layers=4,
                num_heads=4,
                relative_attention_num_buckets=8,
                feed_forward_proj="relu",
                dropout_rate=0.1,
            )
            self.encoder = T5EncoderModel(config)
            self.input_proj = nn.Linear(34, 128)
            self.fc = nn.Linear(128, 256)

        def forward(self, x):
            x = self.input_proj(x)
            attention_mask = torch.ones(x.size()[:2], dtype=torch.long, device=x.device)
            encoder_outputs = self.encoder(inputs_embeds=x, attention_mask=attention_mask)
            pooled = encoder_outputs.last_hidden_state.mean(dim=1)
            x = self.fc(pooled)
            return x

    class MoCapModel(nn.Module):
        def __init__(self):
            super(MoCapModel, self).__init__()
            self.resnet = models.resnet18(pretrained=False)
            self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            self.resnet.fc = nn.Linear(self.resnet.fc.in_features, 256)

        def forward(self, x):
            return self.resnet(x)
    model = CombinedModel()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dic =  torch.load('combined_model.pth', map_location=device)
    model.load_state_dict(dic)
    model.to(device)
    model.eval()
    result_lines = [f"Video name: {tar}\n"]

    for i in range(len(x_train_speech)):
        speech_input = torch.tensor(x_train_speech[i], dtype=torch.float32).unsqueeze(0).to(device)
        mocap_input = torch.tensor(x_train_mocap[i], dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)
        true_label = np.argmax(Y[i]) 

        with torch.no_grad():
            output = model(speech_input, mocap_input)  # [1, 4]
            probabilities = torch.softmax(output, dim=1)
            predicted_index = torch.argmax(probabilities, dim=1).item()
            confidence = probabilities[0, predicted_index].item()

        class_names = ['ang', 'exc', 'neu', 'sad']
        print(ts[i])
        print(f"Predicted: {class_names[predicted_index]} ({predicted_index}), Confidence: {confidence:.2f}")
        print(f"True label: {class_names[true_label]} ({true_label})")



        result_lines.append(f"Time: {"--".join([str(i) for i in ts[i]])}")
        result_lines.append(f"Predicted: {class_names[predicted_index]}, Confidence: {confidence:.2f}")
        result_lines.append(f"True label: {class_names[true_label]} \n")
        emotion_at_time.append(
        {"start": ts[i][0], "end": ts[i][1], "emotion": class_names[predicted_index], "conf": confidence})
    output_text.delete("1.0", tk.END)
    output_text.insert(tk.END, "\n".join(result_lines))
    
        # with open(f"{tar}_results.txt", "w", encoding="utf-8") as file:
        #     file.write("\n".join(result_lines))


current_tar = ""
total_seconds = 0
fps = 25
user_dragging = False

def run_recognition():
    if current_tar:
        f(current_tar)

def select_file():
    global cap, is_paused, current_tar, total_seconds, fps
    file_path = filedialog.askopenfilename(filetypes=[("Video files", "*.mp4 *.avi *.mov")])
    if file_path:
        current_tar = os.path.splitext(os.path.basename(file_path))[0]
        if cap:
            cap.release()
        cap = cv2.VideoCapture(file_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        total_seconds = int(total_frames / fps)
        frame_slider.config(to=total_seconds)
        time_label.config(text="0 / " + str(total_seconds) + " sec")
        is_paused = False
        play_video()

def toggle_pause():
    global is_paused
    is_paused = not is_paused
    if not is_paused:
        play_video()

def on_slider_press(event):
    global user_dragging
    user_dragging = True

def on_slider_release(event):
    global user_dragging
    second = frame_slider.get()
    if cap:
        frame_idx = int(second * fps)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
    user_dragging = False
    if not is_paused:
        play_video()

def play_video():
    global cap, is_paused, fps
    if cap and cap.isOpened() and not is_paused:
        ret, frame = cap.read()
        if ret:
            current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
            second = current_frame / fps
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (640, 360))
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            video_label.imgtk = imgtk
            video_label.config(image=imgtk)
            if not user_dragging:
                frame_slider.set(int(second))
            time_label.config(text=f"{int(second)} / {total_seconds} sec")

            # 更新当前情感显示
            emotion = "none"
            for item in emotion_at_time:
                if item["start"] <= second <= item["end"]:
                    emotion = f"{item['emotion']} ({item['conf']:.2f})"
                    break
            current_emotion_label.config(text=f"emotion: {emotion}")

            video_label.after(33, play_video)
        else:
            cap.release()

root = tk.Tk()
root.title("Player")
root.geometry("1200x700")
root.configure(bg="#f2f2f2")

main_frame = tk.Frame(root, bg="#f2f2f2")
main_frame.pack(fill=tk.BOTH, expand=True, padx=20, pady=20)

video_frame = tk.Frame(main_frame, bg="#f2f2f2")
video_frame.grid(row=0, column=0, padx=10, pady=10)

video_label = Label(video_frame, bg="#000")
video_label.pack()

frame_slider = Scale(video_frame, from_=0, to=100, orient=HORIZONTAL, length=640, showvalue=0)
frame_slider.pack(pady=5)
frame_slider.bind("<ButtonPress-1>", on_slider_press)
frame_slider.bind("<ButtonRelease-1>", on_slider_release)

time_label = Label(video_frame, text="0 / 0 sec", font=("Arial", 12), bg="#f2f2f2")
time_label.pack()

result_frame = tk.Frame(main_frame, bg="#f2f2f2")
result_frame.grid(row=0, column=1, padx=10, pady=10, sticky="n")

Label(result_frame, text="result", font=("Arial", 16), bg="#f2f2f2").pack(pady=5)
output_text = Text(result_frame, height=25, width=50, font=("Courier", 11))
output_text.pack()

control_frame = tk.Frame(root, bg="#f2f2f2")
control_frame.pack(side=tk.BOTTOM, pady=15)

btn_style = {"font": ("Arial", 12), "bg": "#4CAF50", "fg": "white", "padx": 15, "pady": 5}

Button(control_frame, text="choose video", command=select_file, **btn_style).pack(side=tk.LEFT, padx=10)
Button(control_frame, text="play / pause", command=toggle_pause, **btn_style).pack(side=tk.LEFT, padx=10)
Button(control_frame, text="run detection", command=run_recognition, **btn_style).pack(side=tk.LEFT, padx=10)
current_emotion_label = Label(video_frame, text="emotion: none", font=("Arial", 14), fg="blue", bg="#f2f2f2")
current_emotion_label.pack(pady=5)
root.mainloop()

C:\Users\1\AppData\Local\Temp\ipykernel_9104\2906742033.py:26: DeprecationWarning: Please import `dct` from the `scipy.fftpack` namespace; the `scipy.fftpack.realtransforms` namespace is deprecated and will be removed in SciPy 2.0.0.
  from scipy.fftpack.realtransforms import dct


AttributeError: partially initialized module 'torch._inductor' from 'C:\Users\1\miniconda3\Lib\site-packages\torch\_inductor\__init__.py' has no attribute 'custom_graph_pass' (most likely due to a circular import)